In [ ]:
#the measurement update is a Bayesian update to the prior
#based on the measurment

def update(mean1, var1, mean2, var2):
    new_mean = (var2*mean1 + var1*mean2)/(var1+var2)
    new_var = 1/(1/var2 + 1/var1)
    return new_mean, new_var


#implement motion.  If we give a control value u, predict output
#the motion update is a convolution

def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2
    new_var = var1 + var2
    return new_mean, new_var


measurements = [5., 6., 7., 9., 10.]
motions = [1.,1.,2.,1.,1.]
measurement_sigma = 4
motion_sigma = 2
mu = 5.
sig = 10000

for measurement, motion in zip(measurements, motions):
  #measurement update
  mu, sig = update(mu, sig, measurement, measurement_sigma)
  print("Update: ", mu,sig)
  #motion update
  mu, sig = predict(mu, sig, motion, motion_sigma) 
  print("Predict: ", mu,sig)

In [ ]:
# Ultimately, we want to calculate the state, x, of the system

x(k) = Ax(k-1) + Bu(k-1) + w(k-1), given Z(k) = Hx(k) + v(k)  

w and k are known as process and measurment noise -- represented by Q 
process noise covariance matrix and R the measurement noise covariance matrix
A is an nxn state at the previous time step (k-1) to the state of the current state
B is an nx1 matrix relateing the option control input u to the current state
z(k) - Hx(k) is the residual or measurement innnovaton; a diff or zero means that the
estimate and the measurement are equal.  

n x m matrix K is chosen to be gain or blending factor that minimizes the 
posterior error covariance.

K(k) = P(k)H.t(HPkH.t + R)-1  = Pk * H.t/(HP(k)H.t + R
                                         )
k is discrete time

In [37]:
import numpy as np
import matplotlib.pyplot as plt

In [84]:
##step 1/7; 
def update_prediction():
    global x,u,A,B
    print("dot(A,x)", np.dot(A,x))
    print("dot(B,u)", np.dot(B,u))
    
    x = np.add(np.dot(A,x), np.dot(B,u))
    ##not sure why we get x with dim 2x2 
    
    #x = x[0]
    print("x after update: ", x)


In [74]:
def initial_process_cov_matrix():
    global P
    #initial process covariance matrix
    ##do this once
    #delta Pk = 20, 20**2 = 400
    #delta Pvk = 5; 5**2 = 25
    #20*5 cross terms
    P = np.array([[400, 100], [100, 25]])
    ##however, you can usually eliminate the off diagonals
    P[0,1] = 0
    P[1,0] = 0
    #P = np.array([[400, 0], [0, 25]])

    print("Initial Process Covariance Matrix:", P)


In [75]:
#step 3/7; update Process Covarience Matrix
def update_predicted_process_cov_matrix():
    global A, P, Q
    #update the Process Covariance Matrix
    Q = 0.  ##reflects the error in calculating process cov matrix (not sure what)
    P = np.dot(np.dot(A, P), A.T) + Q

    #drop off diagnonals 
    P[0,1] = 0
    P[1,0] = 0
    print("Predicted Process Covariance Matrix: ", P)

In [76]:
def calculate_Kalman_gain():
    global R, P, H, K
    ##step 4/7 calculate the Kalman Gain
    ##use observation errors and square them
    ##delta x is 25**2=625, delta vx = 6**2=36
    S = R + np.dot(H, np.dot(P, H.T))
    K = np.dot(np.dot(P, H.T), np.linalg.inv(S))
    print("Kalman Gain Matrix: ", K)


In [77]:
def import_new_observation(m):
    global Y
    #Step 5/7
    ##import the new observation
    #4260, 282
    #m = [4260, 282]
    #Observed state Yk = CYkm + Zk
    C = np.array([[1,0],[0,1]])
    measurement = np.array(m)
    Y = np.dot(C, measurement)
    print("observed: ", Y)




In [78]:
def update_current_state():
    global Y, H, x
    #step 6/7

    ##current state
    #Xk = Xkp + K[Yk - HXkp]
    delta = Y - np.dot(H, x)
    print("delta: ", delta)
    x = x + np.dot(K, delta)
    print("adjusted x:", x)



In [79]:
def update_process_cov_matrix():
    global K, H, O, R, P
    #step 7/7 update Process Covarance Matrix
    #Pk = (I-KH)*Pkp

    I = np.eye(n)
    P = np.dot(np.dot(I - np.dot(K, H), P), 
                (I - np.dot(K, H)).T) + np.dot(np.dot(K, R), K.T)   

    print("Updated Process Covariance Matrix: ", P)


In [85]:
##initialize the matrices

dt = 1/60.
dt=1.

##2D case
#A is the state transition matrix
A = np.array([[1,dt], [0,1]])
n = A.shape[1]

x = np.array([4000,280])
print(np.shape(x))
#x is the state matrix--tells position and velocity of object to track
#u is the control matix--adds in the effects of acceleration
u = 2 #2 m/s**2

#B is the change in position due to acceleration
#B is a 2x1
##use [:,None]  to make b a column vector (.T also works)
B = np.array([.5*(dt**2),dt])

#R contains observation errors [delta x = 25m, delta v = 6m/s]
#these are in std; square them to get a proper R
R = np.array([[625, 0], [0, 36]])
    
H = np.array([[1, 0],[0,1]])
Q = 0
#Q = np.array([[0.05, 0.0], [0.05, 0.05, 0.0], [0.0, 0.0, 0.0]])

P = np.eye(n)

(2,)


In [86]:
#Vinit = 250 m/s. ; x = 400 m
#process error 20m, 5m/s error


pos_measurements = [4260, 4550, 4860, 5110]
vel_measurements = [282, 285, 286, 290]

initial_process_cov_matrix()

for position, velocity in zip(pos_measurements, vel_measurements):
    print("************NEW VALUES******************")
    update_prediction()
    update_predicted_process_cov_matrix()
    calculate_Kalman_gain()
    import_new_observation ([position, velocity])
    update_current_state()
    update_process_cov_matrix()
    
    
    

Initial Process Covariance Matrix: [[400   0]
 [  0  25]]
************NEW VALUES******************
dot(A,x) [4280.  280.]
dot(B,u) [1. 2.]
x after update:  [4281.  282.]
Process Covariance Matrix:  [[425.   0.]
 [  0.  25.]]
Kalman Gain Matrix:  [[0.4047619  0.        ]
 [0.         0.40983607]]
observed:  [4260  282]
delta:  [-21.   0.]
adjusted x: [4272.5  282. ]
Updated Process Covariance Matrix:  [[252.97619048   0.        ]
 [  0.          14.75409836]]
************NEW VALUES******************
dot(A,x) [4554.5  282. ]
dot(B,u) [1. 2.]
x after update:  [4555.5  284. ]
Process Covariance Matrix:  [[267.73028884   0.        ]
 [  0.          14.75409836]]
Kalman Gain Matrix:  [[0.29990053 0.        ]
 [0.         0.29069767]]
observed:  [4550  285]
delta:  [-5.5  1. ]
adjusted x: [4553.85054707  284.29069767]
Updated Process Covariance Matrix:  [[187.4378327    0.        ]
 [  0.          10.46511628]]
************NEW VALUES******************
dot(A,x) [4838.14124475  284.29069767]
do

In [ ]:
print(P)
print(x)

In [ ]:
   

  #K is the Kalman Grain; strikes a balance measurement and prediction
  #First calculate S which is the denominator of the Kalman gain
  #H is usually an identity matrix H.T is the same as H because I.T = I
  #R is the same form as the covariance matrix with delta(x) and delta(v) on diagonals

S = R + np.dot(H, np.dot(P, H.T))
K = np.dot(np.dot(P, H.T), np.linalg.inv(S))


  ##delta is the different between prediction & measurements
  #H is the observation model. (np.dot(H,x) is the estimate


    
  #Q is the Processes Noise Covariance Matrix
  #Prevents the State Covariance Matrix from -> 0
    
  #R is the Measurement Covariance Matrix (error in measurement)

  #If R-> 0, then k-> 1 (weighted toward measurement)
  #if R -> large, then k -> 0 (weighted toward prediction)
  #If P -> 0 then measurments updates are mostly ignored
    
    

  I = np.eye(n)

  P = np.dot(np.dot(I - np.dot(K, H), P), 
            (I - np.dot(K, H)).T) + np.dot(np.dot(K, R), K.T)    
  #x = np.dot(F,x)
 
plt.plot(x[0])
plt.show()

In [ ]:
#copy of Dr. B 2d example

dt = 1/60.
dt=1.
F = np.array([[1, 0, dt, 0], [0, 1, 0, dt], [0, 0, 1, 0], [0, 0, 0, 1]])
x = np.array([10, 10, 5, 5])
#print(np.shape(x))
#x is the state matrix--tells position and velocity of object to track
#u is hte control matix--adds in the effects of acceleration

#B = np.array([.5*(dt**2),dt])[:,None]
B = np.array([[.5*(dt**2), 0],[0, .5*(dt**2)], [dt, 0], [0, dt]])
print(np.shape(B))
##use [:,None]  to make b a column vector (.T also works)
#B = np.array([.5*(dt**2),dt])[:,None]
#B = 0
#Ax + Bu  -- A is the change in position due to velocity
#B is the change in position due to acceleration
#B is a 2x1

u = [1,1]
n = F.shape[1]

print(n)

H = np.array([1, 0]).reshape(1, 2)
Q = 0
#Q = np.array([[0.05, 0.0], [0.05, 0.05, 0.0], [0.0, 0.0, 0.0]])
R = np.array([0.5]).reshape(1, 1)
P = np.eye(n) 

m = np.linspace(-10, 10, 100)
measurements = - (m**2 + 2*m - 2)  + np.random.normal(0, 2, 100)
#Yk = CXk+ zk;;; ys are measrements [position, velocity] or just position
for measurement in measurements:
  #this is the Estimate with no noise--the predict function
 
  x = np.dot(F,x) + np.dot(B,u)
  print(x)
  P = np.dot(np.dot(F, P), F.T) + Q
  print(P)
  ##delta is the different between prediction & measurements  
#  delta = measurement - np.dot(H, x)
#  print("delta: ", delta)

#  S = R + np.dot(H, np.dot(P, H.T))
#  print("S:", S)
 
  #K is the Kalman Grain; strikes a balance measurement and prediction
#  K = np.dot(np.dot(P, H.T), np.linalg.inv(S))
#  x = x + np.dot(K, delta)
#  I = np.eye(n)

#  P = np.dot(np.dot(I - np.dot(K, H), P), 
#            (I - np.dot(K, H)).T) + np.dot(np.dot(K, R), K.T)    
  #x = np.dot(F,x)
#  print(x[0])
#  print(P)
#  print(np.shape(x))

    
#plt.plot(x[0])
#plt.show()

In [ ]:
print(np.dot(B,u))


In [ ]:
test = np.linspace(-10, 10, 100)
measurements = - (test**2 + 2*test - 2)  + np.random.normal(0, 2, 100)

plt.plot(range(len(measurements)), measurements, label = 'Measurements')
plt.show()

In [ ]:
import numpy as np

class KalmanFilter(object):
    def __init__(self, F = None, B = None, H = None, Q = None, R = None, P = None, x0 = None):

        if(F is None or H is None):
            raise ValueError("Set proper system dynamics.")

        self.n = F.shape[1]
        self.m = H.shape[1]

        self.F = F
        self.H = H
        
        self.B = 0 if B is None else B
        self.Q = np.eye(self.n) if Q is None else Q
        self.R = np.eye(self.n) if R is None else R
        self.P = np.eye(self.n) if P is None else P
        self.x = np.zeros((self.n, 1)) if x0 is None else x0

        
    def predict(self, u = 0):
        self.x = np.dot(self.F, self.x) + np.dot(self.B, u)
        print(np.shape(x))
        self.P = np.dot(np.dot(self.F, self.P), self.F.T) + self.Q
        return self.x

    def update(self, z):
        y = z - np.dot(self.H, self.x)
        S = self.R + np.dot(self.H, np.dot(self.P, self.H.T))
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))
        self.x = self.x + np.dot(K, y)
        I = np.eye(self.n)
        self.P = np.dot(np.dot(I - np.dot(K, self.H), self.P), 
            (I - np.dot(K, self.H)).T) + np.dot(np.dot(K, self.R), K.T)

        
        
        
def example():
    dt = 1.0/60
    F = np.array([[1, dt, 0], [0, 1, dt], [0, 0, 1]])
    H = np.array([1, 0, 0]).reshape(1, 3)
    Q = np.array([[0.05, 0.05, 0.0], [0.05, 0.05, 0.0], [0.0, 0.0, 0.0]])
    R = np.array([0.5]).reshape(1, 1)

    x = np.linspace(-10, 10, 100)
    measurements = - (x**2 + 2*x - 2)  + np.random.normal(0, 2, 100)


    kf = KalmanFilter(F = F, H = H, Q = Q, R = R)
    print(kf)
    predictions = []


    for z in measurements:
        predictions.append(np.dot(H,  kf.predict())[0])
        kf.update(z)

    import matplotlib.pyplot as plt
    plt.plot(range(len(measurements)), measurements, label = 'Measurements')
    plt.plot(range(len(predictions)), np.array(predictions), label = 'Kalman Filter Prediction')
    plt.legend()
    plt.show()

if __name__ == '__main__':
    example()